# Gabriel Bertasius & Jaden Ford#

# Predicting Game Success: A Regression Analysis on the Steam Games Dataset #

In [ ]:
import numpy as np
import pandas as pd
# show all columns
pd.set_option('display.max_columns', None)

## Downloading and loading data

In [ ]:
# load the data into a dataframe for easy handling
import os
from datetime import datetime
import pickle
import gzip
DATASET_DIR = './data/'
DATASET_FILENAME = 'steamgames.parquet'
DATASET_PATH = DATASET_DIR+DATASET_FILENAME
DATASET_COMPRESSION = 'zstd'  # Very fast and compresses as well as gzip
MODELS_DIR = './models/'
MODELS_FILENAME = 'model-'
download_data = 1


def check_file_exists(path: str) -> bool:
    return os.path.exists(path)


def check_data_dir_exists() -> bool:
    return os.path.exists(DATASET_DIR)

def check_models_dir_exists() -> bool:
    return os.path.exists(MODELS_DIR)

def create_data_dir():
    directory_name = DATASET_DIR
    try:
        os.mkdir(directory_name)
        print(f"Directory '{directory_name}' created successfully.")
    except FileExistsError:
        print(f"Directory '{directory_name}' already exists.")
    except PermissionError:
        print(f"Permission denied: Unable to create '{directory_name}'.")
    except Exception as e:
        print(f"An error occurred: {e}")

def create_models_dir():
    directory_name = MODELS_DIR 
    try:
        os.mkdir(directory_name)
        print(f"Directory '{directory_name}' created successfully.")
    except FileExistsError:
        print(f"Directory '{directory_name}' already exists.")
    except PermissionError:
        print(f"Permission denied: Unable to create '{directory_name}'.")
    except Exception as e:
        print(f"An error occurred: {e}")

def download_steamgames_dataset() -> pd.DataFrame:
    df = pd.read_parquet(
        "hf://datasets/FronkonGames/steam-games-dataset/data/train-00000-of-00001-e2ed184370a06932.parquet")
    return df


def write_dataset_pqt(df: pd.DataFrame, filename: str = DATASET_FILENAME, overwrite: bool = False) -> bool:
    dir = DATASET_DIR
    path = dir+filename
    if (check_data_dir_exists() == False):
        create_data_dir()
    if check_file_exists(path) and overwrite == False:
        print("File exists. Pass 'overwrite' to replace.")
        return False
    else:
        df.to_parquet(path, compression='zstd')
        return True


def read_dataset_pqt(filename: str = DATASET_FILENAME):
    path = DATASET_DIR+filename
    if check_file_exists(path):
        print("Loading dataset from local storage...")
        prq = pd.read_parquet(path)
        print("✅ Local dataset loaded.")
        return prq
    else:
        print("Parquet file not found.")

def datestamp():
    """ Get the current datestamp """
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

def write_model_log(path:str, config: dict, **kwargs):
    with open(path+".txt", "a") as file:
        file.write(f"[{datestamp()}]\n")
        file.write(f"{config}\n")
        if kwargs:
            for x in kwargs:
                file.write(f"{x}\n")
    
def pickle_model(filename: str, model, params_dict:dict, param_grid:dict = None,overwrite: bool=False, **extra_data):
    dir = MODELS_DIR 
    path = dir+filename
    for s in params_dict.values():
        path += f'-{s}'
    if (check_models_dir_exists() == False):
        create_models_dir()
    if check_file_exists(path) and overwrite == False:
        print("File exists. Pass 'overwrite' to replace.")
        return False
    else:
        if param_grid is not None:
            write_model_log(path, param_grid, **extra_data)
        else:
            write_model_log(path, params_dict)
        level = 7   # Good balance between speed and compression
        with gzip.open(path+".pkl.gz", "wb", compresslevel=level) as file:
            pickle.dump(model, file, protocol=5)
        return True

def unpickle_model(filename):
    path = MODELS_DIR+filename
    with gzip.open(path+".pkl.gz", "rb") as file:
        return pickle.load(file)
    
def download_and_save_dataset(force: bool = False, filename: str = DATASET_FILENAME) -> pd.DataFrame | None:
    dir = DATASET_DIR
    path = dir+filename
    if (check_file_exists(path)):
        print(f"⚠️ Dataset exists locally. Path:{path}")
        if (force == False):
            print("Use force=True to download and overwrite.")
            return None
        else:
            print("Redownloading and Overwriting...")
    else:
        print(f"Downloading and saving dataset to {path} ")
    df = download_steamgames_dataset()
    write_dataset_pqt(df, overwrite=True)
    print("✅ Done.")
    print(f"Saved to: {path}")
    return df


df = download_and_save_dataset(force=False)
if(df is None):
    df = read_dataset_pqt()

In [ ]:
# Check for any missing values
sum = df.isnull().sum()
sum[sum != 0]

In [ ]:
# remove any columns that won't contribute to a game's success rating
cols_to_remove = ['About the game', 'Supported languages', 'Full audio languages',
                  'Header image', 'Website', 'Support url', 'Support email', 'Metacritic url',
                  'Score rank', 'Screenshots', 'Movies']
df = df.drop(columns=cols_to_remove, axis=1)
df.head()

In [ ]:
from dataclasses import dataclass, fields, field

@dataclass
class DataMinMax:
    data:dict = field(default_factory=dict)


In [ ]:
# function that calculates the number of years since a game's release date
from datetime import datetime
def years_since_release(date_string):
  if len(date_string) == 11 or len(date_string) == 12:
        date = datetime.strptime(date_string, "%b %d, %Y")
  else: # length must be 8 or 9
      date = datetime.strptime(date_string, "%b %Y")

  current_date = datetime.now()
  years = (current_date - date).days / 365
  return years

# function to return the avg number of estimated owners
def est_owners(num_owners):
  numbers = num_owners.split('-')
  return (int(numbers[0]) + int(numbers[1])) / 2

# function to normalize a numerical column between 0-1 based on min and and max values
def min_max_normalize(column):
  column = np.array(column)
  norm_col = ( column - np.min(column) ) / ( np.max(column) - np.min(column) )
  return norm_col

In [ ]:
# convert release date to years since release
df['Release date'] = df['Release date'].apply(years_since_release)

# return middle value for each given range of estimated owners
df['Estimated owners'] = df['Estimated owners'].apply(est_owners)

# convert windows, mac, and linux columns from boolean to integer
df['Windows'] = df['Windows'].astype(int)
df['Mac'] = df['Mac'].astype(int)
df['Linux'] = df['Linux'].astype(int)

In [ ]:
# Filter out any games that are free, have no peak ccu, and no estimated owners
# This allows us to judge success based on games that competed in certain markets, and have had actual people play them
no_peak_ccu_cols = df[df['Peak CCU'] == 0].index
df = df.drop(no_peak_ccu_cols, axis=0)

no_est_owners_cols = df[df['Estimated owners'] == 0].index
df = df.drop(no_est_owners_cols, axis=0)

no_price_cols = df[df['Price'] == 0].index
df = df.drop(no_price_cols, axis=0)

In [ ]:
# keep a copy of pre_normalized values

df_orig = df.copy(deep=True)

In [ ]:
# normalize any large value ranges
cols_to_normalize = ['Release date', 'Estimated owners', 'Peak CCU', 'Required age', 'Price', 'DLC count',
                     'Metacritic score', 'User score', 'Positive', 'Negative', 'Achievements',
                     'Recommendations', 'Average playtime forever', 'Average playtime two weeks',
                     'Median playtime forever', 'Median playtime two weeks']
for col in cols_to_normalize:
  df[col] = min_max_normalize(df[col])

In [ ]:
# If we want to remove rows that have no reviews, we would have 4269 examples
#df = df.dropna(axis=0, subset='Reviews')
#print(df.shape[0])
#df.isnull().sum()

In [ ]:
print(df.shape)
df.head()

### Counting unique words in Categories, Genres, Tags

'Dumb counting' as in the tags 'turn-based' and 'turn-based combat' or 'turn-based strategy' are different words. These should be ok for word2vec as they're similar.

In [ ]:
df.columns
df['Tags']

def count_unique_words(df, label:str):
    lists:pd.Series= df[label].str.casefold().str.split(',')
    words = set()
    [words.update(x) for x in lists if x is not None]
    print(f"Number of unique {label}: {len(words)}")
    return words

count_unique_words(df, 'Categories') # 39
count_unique_words(df, 'Genres') # 27
count_unique_words(df, 'Tags') # 444
;

In [ ]:
encoded_categories = df['Categories'].str.get_dummies(sep=',')
encoded_genres = df['Genres'].str.get_dummies(sep=',')

df = pd.concat([df, encoded_categories, encoded_genres], axis=1)
df = df.drop(columns=['Categories', 'Genres'], axis=1)

In [ ]:
print(df.shape)
df.head()

### Word2Vec embedding for Tags feature

Currently the embedding for the tags is an average of the tags for a given game. This results in d-dimensional feature embedding where d is the numer of dimensions specified in word2vec training.

todo: process hyphenated and multi-word tags. Treat as one phrase by subbing dashes and spaces with an underline

todo: tuning: what do the parameters do? what can be tweaked? what is desired?

todo: CBOW vs CSkipGram

In [ ]:
label = 'Tags'
lists:pd.Series= df[label].str.casefold().str.split(',')
# lists.fillna('none')
lists = lists.apply(lambda x: ['none'] if x is None else x)
sentences = [x for x in lists]

In [ ]:
print(sentences[9])

In [ ]:
import gensim

model_name = "100features_1minwords_10context"
model = None

num_features = 100  # Word vector dimensionality
min_word_count = 1  # Minimum word count
num_workers = 8  # Number of threads to run in parallel
context = 10  # Context window size
downsampling = 1e-3  # Downsample setting for frequent words

def init_sims(model):
    # If you don't plan to train the model any further, calling
    # init_sims will make the model much more memory-efficient.
    print("get_mean_vector is deprecated. Use get_vector(key, norm=True) instead")
    model.init_sims(replace=True)


if check_file_exists(model_name):
    """Normalize Vectors"""
    print("Loading saved model")
    model = gensim.models.Word2Vec.load(model_name)
    init_sims(model)

else:

    # def process_tags(df: pd.DataFrame):
    # model = gensim.models.Word2Vec

    # Code from:

    # https://www.kaggle.com/competitions/word2vec-nlp-tutorial/overview
    # Set values for various parameters

    print("Training model...")
    model = gensim.models.Word2Vec(
        sentences,
        workers=num_workers,
        vector_size=num_features,
        min_count=min_word_count,
        window=context,
        sample=downsampling,
    )

    # It can be helpful to create a meaningful model name and
    # save the model for later use. You can load it later using Word2Vec.load()
    model.save(model_name)

Vector of the tag 'singleplayer'

In [ ]:
print(len(model.wv.index_to_key))
print(model.wv.index_to_key[3])
model.wv['action']

Word relative rank (cosine similarity)

In [ ]:
model.wv.most_similar('action', topn=10)
model.wv.similar_by_word('action', topn=10) # same result

In [ ]:
type(model)
words = model.wv.index_to_key
words[0:10]
model.wv.most_similar('none') # this needs fixin

In [ ]:
"""
    pre-normalizing will discard sentence length information
    this should ignore differences in numbe of tags specified for each game
    Pre-normalize doesnt matter if init_sims(replace=True) since it will
    precompute normalized vectors.
    Not clear what the point of post_normalize is. May be/not good for training
    the regression model down the line.
"""

tags_vectors = [
    model.wv.get_mean_vector(game, pre_normalize=False, post_normalize=False)
    for game in sentences
]

In [ ]:
print('Number of games', len(tags_vectors))
tags_vectors[0]

In [ ]:
w2vdf = pd.DataFrame(tags_vectors)
assert w2vdf.shape[1] == num_features
w2vdf.columns = [f'w2v_embed_{i}' for i in range(num_features)]
w2vdf.head()

#### Dropping Tags columns and merging embeds

In [ ]:
""" Trick to prevent this from executing twice """
try:
    check_if_w2vdf_already_concat
except NameError:
    df.drop(columns=['Tags'])
    df.reset_index(drop=True, inplace=True)
    df = pd.concat([df, w2vdf], axis=1)
    check_if_w2vdf_already_concat = 1

# del check_if_w2vdf_already_concat

In [ ]:
df.shape

## Train/test data extraction + Regression model selection

The most important metrics when determinng a game's success include the number of estimated owners, peak ccu, number of pos/neg reveiws, and price.

In [ ]:
y = np.array(df[['Estimated owners', 'Peak CCU', 'Positive', 'Negative', 'Price']])
X = np.array(df.drop(columns=['AppID', 'Name', 'Estimated owners', 'Peak CCU', 'Positive', 'Negative', 'Price', 'Reviews', 'Notes', 'Developers', 'Publishers', 'Tags'], axis=1))

print(X.shape)
print(y.shape)

In [ ]:
#print(X[0,:]) # ensure all data is numerical

RandomForestRegressor is used to handle non-linear relationships between a game and the metrics we are predicting. MultiOutputRegressor provides easier setup for the model.

A grid search will also be done on the hyperparemeters for the random forest regressor.

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# 70% training data, 30% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

Disclaimer, this cell takes hours to complete!

In [ ]:
perform_search = False
grid_search = None
if perform_search == True:
      # perform a grid search on hyperparameters for random forest
      # -1 to utilize all processors and speed up training time
      rf = RandomForestRegressor(random_state=42, n_jobs=-1)
      model = MultiOutputRegressor(rf, n_jobs=-1)

      param_grid = [
      {'estimator__n_estimators': [20, 50, 100, 150, 200, 250],
       'estimator__max_features': [1, 20, 'sqrt', 50, 70, 90, 110],
       'estimator__max_depth': [None, 10, 20, 30, 40, 50]}
      ]

      grid_search = GridSearchCV(model, param_grid, n_jobs=-1)
      grid_search.fit(X_train, y_train)

      pickle_model("rf_gridsearch_obj", grid_search, grid_search.best_params_, param_grid[0])
      print(grid_search.best_params_)

*Outdated*\
Best hyperparams were no max branch depth, 50 features, and 100 estimators for random forest. These parameters are the most infuential to model capacity, generalization, and computation. Other parameters like min_samples_split were ommitted from grid search since the default is adequte to recognize patterns in the data.

In [ ]:
from sklearn.metrics import r2_score

if perform_search == True:
    model = grid_search.best_estimator_
    y_pred = model.predict(X_test)

    mse = mean_squared_error(y_test, y_pred, multioutput='raw_values')
    rmse = np.sqrt(mse)
    r2_score_values= r2_score(y_test, y_pred, multioutput='raw_values')

    # ['Estimated owners', 'Peak CCU', 'Positive', 'Negative', 'Price']
    print("Test set Mean Squared Error:", mse)
    print("Test set Root Mean Squared Error:", rmse)
    print("Test set R2 Score:", r2_score_values)

In [ ]:
# load saved model
""" You can download from link in README """
loaded_grid = unpickle_model("rf_gridsearch_obj-50-70-150")

In [ ]:
loaded_grid.best_params_

In [ ]:
predict_labels = ["Estimated owners", "Peak CCU", "Positive", "Negative", "Price"]
calc: DataMinMax = DataMinMax()
for i, label in enumerate(predict_labels):
    calc.data[label] = {
        "min": df_orig[label].min(),
        "max": df_orig[label].max(),
        "r2": r2_score_values[i],
        "rmse": rmse[i],
    }
# owners = df_orig['Estimated owners']
# calc.owners = (min())
# du

In [ ]:
# Convert the data into a DataFrame
rows = []
for label in predict_labels:

    fmt_int = lambda x: f"{x:,.0f}"
    fmt_float = lambda x, precision=2: f"{x:,.{precision}f}"
    min_val = calc.data[label]["min"]
    max_val = calc.data[label]["max"]
    rmse_val = calc.data[label]["rmse"]
    range_val = max_val - min_val
    range_percent = rmse_val * 100
    prediction = rmse_val*range_val
    rows.append({
        "Metric": label,
        "Prediction": fmt_float(prediction, 2),
        "Min": fmt_float(min_val,2),
        "Max": fmt_float(max_val),
        "RMSE": fmt_float(rmse_val, 4),
        "Range (%)": fmt_float(range_percent,2),
    })

# Create DataFrame
df_result = pd.DataFrame(rows)

# Display the DataFrame
df_result

Test set Mean Squared Error: [5.53550695e-05 1.01822779e-05 3.49412680e-05 1.67333804e-04
 1.09633784e-03]

Test set Root Mean Squared Error: [0.0074401  0.00319097 0.00591111 0.01293576 0.03311099]

Test set R2 Score: [0.76685888 0.31981091 0.94586848 0.62299915 0.46089451]

Looking at the R2 Score, which indicates the proportion of variance in the dependent variable that is predictable from the independent variables, the model is able to capture underlying patterns decently for the estimated owners, positive number of reviews, and negative number of reviews. This suggests that relationships between the features and target variables are relatively strong, making them easier to predict.

This is logical. Game characteristics like developers, publishers, and categories will directly influence price and peak ccu  counts more so than the other target variables. Since these aren't taken into account during training to avoid too many feature encodings, the correlation between these characteristics makes them harder to predict. **This will help us assign a score to each prediction when defining a success rating.**

Using the equation: **original_value = (normalized_value * (data_max - data_min)) + data_min**, let's calculate the rmse for each predicted value in the original data scale to gauge how good/bad the mse is.

Original Max Values:                
*   75,000,000 owners                      
*   872,138 CCU                            
*   964,983 Positive Reviews                 
*   138,530 Negative Reviews                    
*   $ 269.99                             


Original Min Values:
*   10,000 owners
*   1 CCU
*   0  Positive Reviews
*   0  Negative Reviews
*   $ 0.6456165345712968

**range % determined by rsme/(max-min)**

RMSE in original data range:
*   567,933.0055858027 owners  (0.76% of the range)
*   2,783.9614091764875 CCU (0.32% of range)
*   5,704.1244660345665  Positive Reviews (0.59% of range)
*   1,791.9903878277946  Negative Reviews (1.3% of range)
*   $ 9.278048072328264 (3.4% of range)